In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore, percentileofscore
import math
import os

In [4]:
BASE_DIR = os.path.join('..', 'data')
RAW_DIR = os.path.join(BASE_DIR, 'raw')
ENGINEERED_DIR = os.path.join(BASE_DIR, 'engineered')
SEASON_DIR = os.path.join(ENGINEERED_DIR, '2023')
PLAYERS_DIR = os.path.join(SEASON_DIR, 'players_stats')

In [9]:
stats_df = {}

for f in os.listdir(PLAYERS_DIR):
    filepath = os.path.join(PLAYERS_DIR, f)
    df = pd.read_csv(filepath)
    stats_df[f.split('.')[0]] = df

stats_df

{'advanced goalkeeping':      Rk              Player Nation Pos                   Squad  \
 0     1             Alisson    BRA  GK               Liverpool   
 1     2     Alphonse Areola    FRA  GK                West Ham   
 2     3   Kepa Arrizabalaga    ESP  GK             Real Madrid   
 3     4        Noah Atubolu    GER  GK                Freiburg   
 4     5   Francisco Barbosa    ESP  GK                   Betis   
 ..   ..                 ...    ...  ..                     ...   
 536  45       Gaspar Servio    ARG  GK            Rosario Cent   
 537  46  Matías Tagliamonte    ARG  GK             Racing Club   
 538  47     Ezequiel Unsain    ARG  GK          Defensa y Just   
 539  48    Manuel Vicentini    ARG  GK  Club Atlético Belgrano   
 540  49        Guido Villar    ARG  GK        Barracas Central   
 
                    Comp     Age  Born   90s  Goals_GA  ...  Goal Kicks_AvgLen  \
 0    eng Premier League  31-098  1992  18.0      0.83  ...               34.5   
 1    

In [10]:
PLAYERS_DICT = [
        "standard",
        "defensive",
        "goal and shot creation",   
        "misc",
        "pass types",
        "passing",
        "playing time",
        "possession",
        "shooting",
        "advanced goalkeeping",
        "goalkeeping",
]

standard = stats_df[PLAYERS_DICT[0]]
defensive = stats_df[PLAYERS_DICT[1]]
goal = stats_df[PLAYERS_DICT[2]]
misc = stats_df[PLAYERS_DICT[3]]
pass_types = stats_df[PLAYERS_DICT[4]]
passing = stats_df[PLAYERS_DICT[5]]
playing_time = stats_df[PLAYERS_DICT[6]]
possession = stats_df[PLAYERS_DICT[7]]
shooting = stats_df[PLAYERS_DICT[8]]
adv_goalkeeper = stats_df[PLAYERS_DICT[9]]
goalkeeper = stats_df[PLAYERS_DICT[10]]

In [19]:
standard.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born',
       'Playing Time_MP', 'Playing Time_Starts', 'Playing Time_Min',
       'Playing Time_90s', 'Performance_Gls', 'Performance_Ast',
       'Performance_G+A', 'Performance_G-PK', 'Performance_PK',
       'Performance_PKatt', 'Performance_CrdY', 'Performance_CrdR',
       'Expected_xG', 'Expected_npxG', 'Expected_xAG', 'Expected_npxG+xAG',
       'Progression_PrgC', 'Progression_PrgP', 'Progression_PrgR',
       'Per 90 Minutes_Gls', 'Per 90 Minutes_Ast', 'Per 90 Minutes_G+A',
       'Per 90 Minutes_G-PK', 'Per 90 Minutes_G+A-PK', 'Per 90 Minutes_xG',
       'Per 90 Minutes_xAG', 'Per 90 Minutes_xG+xAG', 'Per 90 Minutes_npxG',
       'Per 90 Minutes_npxG+xAG', 'Player Link_', 'Player ID_', 'Competition'],
      dtype='object')

In [56]:
def join_dfs(left: pd.DataFrame, right: pd.DataFrame):
    columns_to_filter = list(right.columns.difference(left.columns))
    columns_to_filter.append('Player ID_')
    r = right.filter(columns_to_filter).copy()
    return pd.merge(left=left, right=r, how="left", on="Player ID_")

In [64]:
df_1 = join_dfs(standard, defensive)
df_2 = join_dfs(df_1, goal)
df_3 = join_dfs(df_2, misc)
df_4 = join_dfs(df_3, pass_types)
df_5 = join_dfs(df_4, passing)
df_6 = join_dfs(df_5, playing_time)
df_7 = join_dfs(df_6, possession)
df_8 = join_dfs(df_7, shooting)
df_9 = join_dfs(df_8, adv_goalkeeper)
final_df = join_dfs(df_9, goalkeeper)
final_df.drop_duplicates(inplace=True)

In [66]:
final_df.to_parquet(os.path.join(PLAYERS_DIR, 'merged_df.parquet'), index=False)